In [3]:
!pip install johnsnowlabs

  Obtaining dependency information for johnsnowlabs from https://files.pythonhosted.org/packages/07/d9/9a12fd966c608c76eb7ff2e1ba5d2746d98626164038df8ef4564cf85ca8/johnsnowlabs-5.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for spark-nlp==5.0.1 from https://files.pythonhosted.org/packages/d4/5e/ff63a474a678d713f8de5f874105fa3329bd0db5bf6110dd998c3fd927ec/spark_nlp-5.0.1-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 7.7 MB/s eta 0:00:00
  Obtaining dependency information for nlu==4.2.2 from https://files.pythonhosted.org/packages/c4/7d/6e249988e9f78ba118c4f89de492508ea98b5330526267762492aadf1654/nlu-4.2.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 22.4 MB/s eta 0:00:00
  Obtaining dependency information for databricks-api from https://files.pythonhosted.org/packages/d0/cc/6c3f9cd8b2b6c7a45c95b94d334bc51f

In [1]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()


import pandas as pd
import json
import string
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to

spark = nlp.start()


from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pyspark.sql as SQL
from pyspark import keyword_only

from sklearn.metrics import classification_report


🚨 Visual NLP installation seems broken, there was an exception while importing it. It will not be available on the nlp.xx module
You can run  nlp.install(refresh_install=True, force_browser=True)  to re-install latest version. 
Error Message : No module named 'implicits'
Error Trace: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/johnsnowlabs/visual.py", line 18, in <module>
    from sparkocr.databricks import isRunningInDatabricks
  File "/usr/local/lib/python3.8/dist-packages/sparkocr/databricks.py", line 4, in <module>
    from implicits import implicits
ModuleNotFoundError: No module named 'implicits'

📋 Loading license number 0 from /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👌 JSL-Home is up to date! 
👌 Everything is already installed, no changes made


23/08/10 10:37:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/10 10:37:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.0.1, 💊Spark-Healthcare==5.0.1, running on ⚡ PySpark==3.1.2


# RE Discussion

1. Basic training pipeline(training + evaluation) - from 03.4.Resume_RelationExtractionApproach_Training.ipynb
2. Training pipeline(training+ evaluation) with redl models


Questions:
1. During re model training, a NN architecture is specified. Is this common across all non redl models?
2. The class used while training non dl model is not compatible with redl models(see error at  the bottom). What class can be used for redl models?


## Basic training pipeline

### Dataset preperation

In [5]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/i2b2_clinical_rel_dataset.csv

In [2]:
data = spark.read.option("header","true").format("csv").load("i2b2_clinical_rel_dataset.csv")

data = data.select( 'sentence','firstCharEnt1','firstCharEnt2','lastCharEnt1','lastCharEnt2', "chunk1", "chunk2", "label1", "label2",'rel','dataset')


In [3]:
column_map = {
    "begin1": "firstCharEnt1",
    "end1": "lastCharEnt1",
    "begin2": "firstCharEnt2",
    "end2": "lastCharEnt2",
    "chunk1": "chunk1",
    "chunk2": "chunk2",
    "label1": "label1",
    "label2": "label2"
}

# apply preprocess function to dataframe
data = medical.REDatasetHelper(data).create_annotation_column(
    column_map,
    ner_column_name="train_ner_chunks" # optional, default train_ner_chunks
)

In [4]:
# add relation direction

@F.udf(T.StringType())
def encodeRelationDirection(rel, begin1, begin2):
    if rel != "O":
        if begin1 > begin2:
            return "leftwards"
        else:
            return "rightwards"
    else:
        return "both"



data = data.withColumn("rel_dir", encodeRelationDirection("rel", "firstCharEnt1", "lastCharEnt2"))

train_data = data.where("dataset='train'")
test_data = data.where("dataset='test'")

### Loading the pipeline

In [9]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("sentences")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")\

words_embedder = nlp.WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

finisher = nlp.Finisher()\
    .setInputCols(["relations"])\
    .setOutputCols(["relations_out"])\
    .setCleanAnnotations(False)\
    .setValueSplitSymbol(",")\
    .setAnnotationSplitSymbol(",")\
    .setOutputAsArray(False)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ]embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
Download done! Loading the resource.
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ]pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
Download done! Loading the resource.


[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ]dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
Download done! Loading the resource.
[ / ]

[ \ ]

[ | ]

[ / ]

[OK!]


In [10]:
clinical_re_Model = medical.RelationExtractionModel()\
    .pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")

clinical_re_Model.getClasses()

re_clinical download started this may take some time.
[ | ]re_clinical download started this may take some time.
Approximate size to download 6 MB
Download done! Loading the resource.
[ / ]

2023-08-10 09:20:37.672017: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


['TrWP', 'TrNAP', 'TrCP', 'PIP', 'TeCP', 'TeRP', 'TrIP', 'TrAP', 'O']

### Evaluation

In [ ]:
finetune_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    clinical_re_Model,
    finisher
])

In [17]:
result = finetune_pipeline.fit(test_data).transform(test_data)
result_test_df = result.select('rel', 'relations.result').toPandas()


In [19]:

pd_test_results = result_test_df.explode("result").fillna("O")

print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           O       0.60      0.50      0.54       274
         PIP       0.37      0.22      0.27        88
        TeCP       0.00      0.00      0.00        15
        TeRP       0.34      0.84      0.48       116
        TrAP       0.62      0.24      0.35       108
        TrCP       0.20      0.05      0.08        20
        TrIP       0.00      0.00      0.00        11
       TrNAP       0.40      0.22      0.29         9
        TrWP       0.04      0.11      0.06         9

    accuracy                           0.44       650
   macro avg       0.29      0.24      0.23       650
weighted avg       0.48      0.44      0.42       650



### Training

Questions:
1. This was the architecture mentioned for re_clinical. Can the same be used for all other re models?

In [20]:
#from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder= "./tf_graphs"

re_graph_builder = medical.TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

In [21]:
reApproach_finetune = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(50)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setPretrainedModelPath("/root/cache_pretrained/re_clinical_en_2.5.5_2.4_1596928426753")\
    .setОverrideExistingLabels(False)

finetune_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach_finetune,
    finisher
])

In [22]:
rel_model = finetune_pipeline.fit(train_data)

TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 27, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}


2023-08-10 09:30:48.346809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 09:30:49.321881: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2023-08-10 09:30:49.321968: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2023-08-10

relation_extraction graph exported to ./tf_graphs/re_graph.pb


Training 20 epochs
Epoch 1/20	0.18s	Loss: 1.2211732	ACC: 0.29416668
Epoch 2/20	0.02s	Loss: 0.5527558	ACC: 0.035833336
Epoch 3/20	0.01s	Loss: 0.24001111	ACC: 0.07083333
Epoch 4/20	0.01s	Loss: 0.11152315	ACC: 0.18083334
Epoch 5/20	0.01s	Loss: 0.0957636	ACC: 0.32333332
Epoch 6/20	0.01s	Loss: 0.08317436	ACC: 0.34833333
Epoch 7/20	0.01s	Loss: 0.081808046	ACC: 0.33416668
Epoch 8/20	0.01s	Loss: 0.080394045	ACC: 0.33416668
Epoch 9/20	0.01s	Loss: 0.069431186	ACC: 0.32083333
Epoch 10/20	0.01s	Loss: 0.06968339	ACC: 0.27833334
Epoch 11/20	0.01s	Loss: 0.06706407	ACC: 0.31083333
Epoch 12/20	0.01s	Loss: 0.067792274	ACC: 0.35166666
Epoch 13/20	0.01s	Loss: 0.05476083	ACC: 0.3466667
Epoch 14/20	0.01s	Loss: 0.057616215	ACC: 0.36083335
Epoch 15/20	0.01s	Loss: 0.055765167	ACC: 0.4475
Epoch 16/20	0.01s	Loss: 0.05387204	ACC: 0.44416666
Epoch 17/20	0.01s	Loss: 0.06252569	ACC: 0.40666667
Epoch 18/20	0.01s	Loss: 0.050994568	ACC: 0.41083333
Epoch 19/20	0.01s	Loss: 0.040690016	ACC: 0.4575
Epoch 20/20	0.01s	Loss: 

In [24]:
result = rel_model.transform(test_data)


In [25]:
result_test_df = result.select('rel', 'relations.result').toPandas()
pd_test_results = result_test_df.explode("result").fillna("O")


In [26]:
pred = pd_test_results['result']
gt = pd_test_results['rel']

print(classification_report(gt, pred))

              precision    recall  f1-score   support

           O       0.43      0.92      0.59       274
         PIP       0.00      0.00      0.00        88
        TeCP       0.00      0.00      0.00        15
        TeRP       0.78      0.16      0.26       116
        TrAP       0.15      0.03      0.05       108
        TrCP       0.00      0.00      0.00        20
        TrIP       0.20      0.09      0.13        11
       TrNAP       0.00      0.00      0.00         9
        TrWP       0.00      0.00      0.00         9

    accuracy                           0.42       650
   macro avg       0.17      0.13      0.11       650
weighted avg       0.35      0.42      0.30       650



## Training pipeline(training+ evaluation) with redl models

In [6]:
label_map = {'TrIP': '1',
'TrAP': '1',
'TeCP': '1',
'O': '0',
'TrNAP': '1',
'TrCP': '1',
'PIP': '1',
'TrWP': '1',
'TeRP': '1'}

In [7]:
clinical_re_Model = medical.RelationExtractionDLModel()\
    .pretrained("redl_date_clinical_biobert", "en", 'clinical/models')\
    .setInputCols(["train_ner_chunks", "sentences"])\
    .setOutputCol("relations")

clinical_re_Model.getClasses()

redl_date_clinical_biobert download started this may take some time.
[ | ]redl_date_clinical_biobert download started this may take some time.
Approximate size to download 383.1 MB
[ — ]Download done! Loading the resource.
[ \ ]

2023-08-10 10:42:29.433953: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


['0', '1']

In [8]:
train_data = train_data.replace(to_replace=label_map, subset=['rel'])
test_data = test_data.replace(to_replace=label_map, subset=['rel'])

In [9]:
finetune_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    clinical_re_Model,
    finisher
])

In [10]:
result = finetune_pipeline.fit(test_data).transform(test_data)
result_test_df = result.select('rel', 'relations.result').toPandas()

In [13]:
pd_test_results = result_test_df.explode("result").fillna("O")

print(classification_report(pd_test_results["rel"], pd_test_results["result"]))

              precision    recall  f1-score   support

           0       0.39      0.31      0.35       274
           1       0.56      0.65      0.60       376

    accuracy                           0.51       650
   macro avg       0.48      0.48      0.47       650
weighted avg       0.49      0.51      0.49       650



In [14]:
! ls /root/cache_pretrained/

dependency_conllu_en_3.4.4_3.0_1656845289670
embeddings_clinical_en_2.4.0_2.4_1580237286004
ner_oncology_en_4.2.2_3.0_1669306355829
pos_clinical_en_3.0.0_3.0_1617052315327
redl_date_clinical_biobert_en_4.2.4_3.0_1673731277460
sbiobert_base_cased_mli_en_2.6.4_2.4_1606225728763
sbiobertresolve_snomed_findings_aux_concepts_en_3.1.2_3.0_1645879611162
sentence_detector_dl_healthcare_en_3.2.0_3.0_1628678815210


In [17]:
medical.RelationExtractionDLApproach

AttributeError: module 'johnsnowlabs.medical' has no attribute 'RelationExtractionDLApproach'

In [15]:

graph_folder= "./tf_graphs"

re_graph_builder = medical.TFGraphBuilder()\
    .setModelName("relation_extraction")\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"]) \
    .setLabelColumn("rel")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("re_graph.pb")\
    .setHiddenLayers([300, 200])\
    .setHiddenAct("relu")\
    .setHiddenActL2(True)\
    .setHiddenWeightsL2(False)\
    .setBatchNorm(False)

reApproach_finetune = medical.RelationExtractionApproach()\
    .setInputCols(["embeddings", "pos_tags", "train_ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setLabelColumn("rel")\
    .setEpochsNumber(20)\
    .setBatchSize(200)\
    .setDropout(0.5)\
    .setLearningRate(0.001)\
    .setFixImbalance(True)\
    .setFromEntity("firstCharEnt1", "lastCharEnt1", "label1")\
    .setToEntity("firstCharEnt2", "lastCharEnt2", "label2")\
    .setPretrainedModelPath("/root/cache_pretrained/redl_date_clinical_biobert_en_4.2.4_3.0_1673731277460")\
    .setОverrideExistingLabels(False)

finetune_pipeline = nlp.Pipeline(stages=[
    documenter,
    tokenizer,
    words_embedder,
    pos_tagger,
    dependency_parser,
    re_graph_builder,
    reApproach_finetune,
    finisher
])

In [16]:
rel_model = finetune_pipeline.fit(train_data)



TF Graph Builder configuration:
Model name: relation_extraction
Graph folder: ./tf_graphs
Graph file name: re_graph.pb
Build params: {'input_dim': 1149, 'output_dim': 6, 'hidden_layers': [300, 200], 'hidden_act': 'relu', 'hidden_act_l2': True, 'hidden_weights_l2': False, 'batch_norm': False}


2023-08-10 10:44:33.810121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 10:44:34.724435: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2023-08-10 10:44:34.724532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:
2023-08-10

relation_extraction graph exported to ./tf_graphs/re_graph.pb


Py4JJavaError: An error occurred while calling o592.fit.
: java.lang.ClassCastException: com.johnsnowlabs.nlp.annotators.re.RelationExtractionDLModel cannot be cast to com.johnsnowlabs.nlp.annotators.generic_classifier.GenericClassifierModel
	at com.johnsnowlabs.nlp.ParamsAndFeaturesReadable.$anonfun$onRead$1(ParamsAndFeaturesReadable.scala:50)
	at com.johnsnowlabs.nlp.ParamsAndFeaturesReadable.$anonfun$onRead$1$adapted(ParamsAndFeaturesReadable.scala:49)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at com.johnsnowlabs.nlp.ParamsAndFeaturesReadable.onRead(ParamsAndFeaturesReadable.scala:49)
	at com.johnsnowlabs.nlp.ParamsAndFeaturesReadable.$anonfun$read$1(ParamsAndFeaturesReadable.scala:61)
	at com.johnsnowlabs.nlp.ParamsAndFeaturesReadable.$anonfun$read$1$adapted(ParamsAndFeaturesReadable.scala:61)
	at co

In [ ]:
result = rel_model.transform(test_data)
